**Ressources**:

- [TF Tutorial](https://www.tensorflow.org/tutorials/images/data_augmentation)
- [Image Augmentation](https://towardsdatascience.com/image-augmentation-for-deep-learning-using-keras-and-histogram-equalization-9329f6ae5085)

In [ ]:
import pandas as pd
import numpy as np

import os
import pathlib
import shutil
from skimage import io

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split


# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import util
import surf_hog_analysis

SEED = 42

---

### 1. Preparations

In [ ]:
df = pd.read_csv('data/train_complete.csv')

In [ ]:
util.isolate_single_defects(df)

In [ ]:
df = util.add_blackness_attributes(df, 'train_images')

In [ ]:
df.head(2)

In [ ]:
df.to_csv('data/train_single_defects_with_blackness.csv', sep=',', index=False)

In [ ]:
X = df.copy()
y = X.pop('ClassId')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = SEED)

Since oversampling is only applied to the training data, we needed to split our data set.

In [ ]:
def get_indices_for_class_id(y, class_id):
    pos_of_class_id = (y == class_id)
    indices = pos_of_class_id[pos_of_class_id].index.values
    return indices

In [ ]:
class1_images = X_train.loc[util.get_indices_for_class_id(y_train, 1)]
class2_images = X_train.loc[util.get_indices_for_class_id(y_train, 2)]
class3_images = X_train.loc[util.get_indices_for_class_id(y_train, 3)]
class4_images = X_train.loc[util.get_indices_for_class_id(y_train, 4)]

print(f'There are {len(class1_images)} train images for ClassId 1')
print(f'There are {len(class2_images)} train images for ClassId 2')
print(f'There are {len(class3_images)} train images for ClassId 3')
print(f'There are {len(class4_images)} train images for ClassId 4')

In [ ]:
print(f'ClassId 2 correspinds to {len(class2_images) / len(X_train)} % of train images.')

In [ ]:
# temporarily safe all images from `ClassId` 2
path = pathlib.Path.cwd()
try:
    os.mkdir(path.joinpath('data','oversampling_test'))
except:
    print('Images already exist.')
    
# von Michael kopiert für Ordnererstellung basierend auf x_train
for i in range(len(class2_images)):
    origin_train_path = path.joinpath('data', 'train_images')
    source_file = class2_images.iloc[i,1]
    target_directory = path.joinpath('data', 'oversampling_test')
    shutil.copy2(origin_train_path.joinpath(source_file) , target_directory.joinpath(source_file))


After having a look at the images from `ClassId` 2, it becomes clear, that many of them have a high percentage of black pixels, some are entirely black even. It may be useful to delete such images since an augmentation would not really help to strengthen the robustness to identify images of this `ClassId`.

In [ ]:
# class_ids = y_train[util.get_indices_for_class_id(y_train, 2)]
# print_batch(class2_images, class_ids)

In [ ]:
image = io.imread('data/oversampling_test/b963c168c.jpg')
plt.figure(figsize=(18,5))
plt.imshow(image);

In [ ]:
def get_black_columns(image, threshold=5):
    num_columns = 0
    
    for column in range(image.shape[1]):
        color_sum = image[:, column].sum()
        
        if color_sum <= image.shape[0] * 3 * threshold:
            num_columns += 1
            
    return num_columns

In [ ]:
black_columns = util.get_black_columns(image)
black_columns

In [ ]:
plt.figure(figsize=(18,5))
plt.axvline(x=black_columns)
plt.imshow(image);

In [ ]:
def add_blackness_attributes(image_df, folder_extension, class_id):
    black_columns = []
    black_columns_percentage = []

    for image_id in image_df.ImageId:
        image = io.imread('data/' + folder_extension + '/' + image_id)
        black_columns.append(get_black_columns(image))
        black_columns_percentage.append(get_black_columns(image) / image.shape[1])

    temp = pd.DataFrame(list(zip(black_columns, black_columns_percentage)), 
                        index=get_indices_for_class_id(y_train, class_id), 
                        columns = ['BlackColumns', 'PercentageBlack'])
    image_df = pd.merge(image_df, temp, left_index=True, right_index=True)
    #print(image_df)

    return image_df

In [ ]:
#class2_images = util.add_blackness_attributes_for_single_class(class2_images, y_train,'oversampling_test', class_id=2)

In [ ]:
class2_images.describe()

In [ ]:
num_mostly_black = class2_images.query('PercentageBlack >= 0.5').ImageId.count()
print(f'There are {num_mostly_black} pictures (or {num_mostly_black / class2_images.shape[0]} % \
        of total) images that are mostly black (>= 50 %).')

In [ ]:
mostly_black = class2_images.query('PercentageBlack >= 0.5')
mostly_black.PercentageBlack.describe()

In [ ]:
mostly_black

It is striking, that `mostly_black` images have at least 72 % blackness on them (up to 94.5 %).

In [ ]:
def print_batch(df_with_filepath, class_ids, blackness=False, show_keypoints=False, number_images=5):
    # create random index for `number_images`
    random_index = np.array(np.random.rand(2 * number_images) * len(df_with_filepath.ImageId), dtype='int')
    print(random_index)
    # define subplot grid
    fig, axes = plt.subplots(number_images, 2, figsize=(18,8))

    for i in range(number_images * 2):
        # gather required info to retrieve image and label the plots
        file_path_to_image = df_with_filepath['FilePath'].iloc[random_index[i]]
        class_id = class_ids.iloc[random_index[i]]
        image_id = df_with_filepath['ImageId'].iloc[random_index[i]]
        if blackness:
            blackness = df_with_filepath['PercentageBlack'].iloc[random_index[i]]
        if show_keypoints:
            keypoints = df_with_filepath['NumberKP'].iloc[random_index[i]]

        # read-in the image
        img = io.imread(file_path_to_image)
        
        row = i % number_images
        col = int(i // number_images)
        ax_ = axes[row][col]
        ax_.imshow(img)
        #plt.imshow(img)
        
        title = f'Image ID: {image_id} | ClassId: {class_id}'
        if blackness:
            title += f' | Percentage Black: {blackness}'
        if show_keypoints:
            title += f' | Number Keypoints: {keypoints}'
        ax_.set_title(title, fontsize=14);
        
        # adjust distance between subplots
        plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=1, 
                    top=1.2, 
                    wspace=0.2, 
                    hspace=0.2)
        plt.axis("off")

In [ ]:
# Visualize some images of `mostly_black`
class_ids_mb = y_train[util.get_indices_for_class_id(y_train, 2)]
print_batch(mostly_black, class_ids_mb, blackness=True)

In [ ]:
black_image = io.imread('data/oversampling_test/08193cfc8.jpg')
plt.figure(figsize=(18,5))
plt.imshow(black_image);

---

### Analysis of black percentage per class

In [ ]:
class1_images = util.add_blackness_attributes_for_single_class(class1_images, y_train, 'train_images', 1)
class3_images = util.add_blackness_attributes_for_single_class(class3_images, y_train, 'train_images', 3)
class4_images = util.add_blackness_attributes_for_single_class(class4_images, y_train, 'train_images', 4)

In [ ]:
class1_images.describe()

In [ ]:
class3_images.describe()

In [ ]:
class4_images.describe()

---

### Augementation trials

[`tf.image`](https://www.tensorflow.org/api_docs/python/tf/image)

In [ ]:
import os
import albumentations
import cv2
import time
import random

# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import data_preparation_cnn

In [ ]:
def make_folder():
    # prepare folder structure
    try:
        path = os.getcwd()
        temp_path = path + "/data/augmentations"
        os.mkdir(temp_path)
    except:
        print('Folder already exists.')

In [ ]:
import albumentations as A

augment = A.Compose([
    #A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    # A.OneOf([
    #     A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
    #     A.GridDistortion(p=0.5),
    #     A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)                  
    #     ], p=0.8),
    A.CLAHE(p=0.8),
    A.RandomBrightnessContrast(p=0.8),    
    A.RandomGamma(p=0.8)
])

In [ ]:
def augement_images(image_ids, num_augmentations, class_id):
    print(f'beginning augmentation for ClassId {class_id}...')
    start = time.time()
    
    path = os.getcwd()
    #path_suffix = 'c' + str(class_id) + '/'
    
    target_directory_image = '/data/augmentations/'
    
    aug_ids = []
    class_ids = []
    file_paths = []
    
    i = 1
    
    while i <= num_augmentations:
        #print(i)
        number = random.randint(0, len(image_ids) -1)
        image_id = image_ids[number]
        #print(image_id, mask_id)
        
        aug_ids.append('aug_' + str(i) + '_' + image_id)
        class_ids.append(class_id)
        file_paths.append(path + target_directory_image + image_id)
        
        original_image = cv2.imread('data/train_images/' + image_id)
        #print(original_image)
      
        augmented = augment(image=original_image)
        transformed_image = augmented['image']
        #transformed_mask = augmented['mask']
        
        os.chdir(path + target_directory_image)
        written = cv2.imwrite('aug_' + str(i) + '_' + image_id, transformed_image)
        #print('image written:',written')

        os.chdir(path)
        
        i += 1
    
    temp = pd.DataFrame(list(zip(file_paths,aug_ids, class_ids)), columns=['FilePath','ImageId','ClassId'])
    
    end = time.time()
    print(f'augmented {num_augmentations} images of ClassId {class_id}')
    print('time required for augmentation:', end - start)
    print()
    
    return temp

In [ ]:
df = pd.read_csv('data/train_single_defects_with_blackness.csv')

In [ ]:
df.head(2)

In [ ]:
def create_df_aug(df):
    
    make_folder()
    
    num_images_class_3 = df.groupby('ClassId').count().ImageId[3]
    max_images = num_images_class_3

    # create empty df
    df_aug = pd.DataFrame(columns=['FilePath','ImageId','ClassId'])

    for i in [1,2,3,4]:
        image_ids = df.query('ClassId == @i').ImageId.values

        temp = augement_images(image_ids=image_ids, num_augmentations=max_images, class_id=i)
        df_aug=pd.concat([df_aug, temp], axis=0)

    return df_aug.reset_index(drop=True)

In [ ]:
df_aug = create_df_aug(df)

In [ ]:
df_aug

In [ ]:
df_aug.to_csv('data/train_single_defects_augmented.csv', sep=',', index=False)

#### Write Image

In [ ]:
# # Iterate and see the pictures and labels
# img_batch, labels = next(it)
# image = img_batch[0]
# #print(img_batch)
# plt.imshow(image)
# print(labels[0])
# image = image.numpy() 
# image *= 256
# written = cv2.imwrite(cwd.as_posix() + '/image.jpg', image)
# print(written)